# 1. Decorators 101: Core Concepts

A **decorator** in Python is a **higher-order function** that takes another function (or class) as an argument, wraps it with additional behavior, and returns a new function (or class).

## 1.1 Anatomy of a Basic Function Decorator

In [14]:
def my_decorator(func):
    def wrapper(*args, **kwargs):
        print("Do something before")
        result = func(*args, **kwargs)
        print("Do something after calling function")
        return result
    return wrapper

@my_decorator
def greet_me(name: str) -> str:
    return f"Hello, {name}!"

greet_me("Junaid")

Do something before
Do something after calling function


'Hello, Junaid!'

When you use the **`@my_decorator`** syntax:

- Python passes the function `greet` to `my_decorator`, which returns `wrapper`.
- The name `greet` in your code then references `wrapper`.
- Calling `greet("Alice")` actually calls `wrapper("Alice")`.

### Example 1: Simple Logging Decorator

In [19]:
def log_calls(func):
    """Logger Decorator"""
    def wrapper(*args, **kwargs):
        print(f"[LOG] Calling {func.__name__} with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper

@log_calls
def add(a: int, b: int) -> int:
  """Add """
  return a + b

add(3, 5)
print(f"What about Metadata {add.__doc__}")
# [LOG] Calling add with args=(3, 5), kwargs={}
# [LOG] add returned 8

[LOG] Calling add with args=(3, 5), kwargs={}
[LOG] add returned 8
What about Metadata None



**Key takeaway**: Decorators let you **interject** extra logic (e.g., logging, timing, caching) around function calls.

---

## 1.2 Decorators with Arguments

Sometimes you want a **parametrized decorator**. This requires an extra “factory” function that takes your decorator arguments and returns the real decorator.

### Example 2: Repetition Decorator

In [16]:
def repeat(times: int):
    """A decorator factory that repeats a function call 'times' times."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            result = None
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(times=3)
def greet(name: str):
    print(f"Hello, {name}!")

greet("Alice")
# Hello, Alice!
# Hello, Alice!
# Hello, Alice!

Hello, Alice!
Hello, Alice!
Hello, Alice!


- **`repeat(times=3)`** returns the actual decorator.
- That decorator wraps `greet`, repeating the call 3 times.

---

## 1.3 Preserving Metadata with `functools.wraps`

A subtlety with decorators is that they can overwrite the original function’s metadata (`__name__`, `__doc__`, etc.). Use `@functools.wraps(original_func)` in the wrapper to preserve these.



In [20]:
import functools

def logger(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        """Wrapper docstring"""
        print(f"Calling {func.__name__}...")
        return func(*args, **kwargs)
    return wrapper

@logger
def example_function(x: int) -> int:
    """Original docstring of example_function."""
    return x * 2

print(example_function.__name__)  # "example_function"
print(example_function.__doc__)   # "Original docstring of example_function."

example_function
Original docstring of example_function.


---

# 2. Decorators with DataClasses

Data classes (`@dataclass`) automatically generate special methods like `__init__`, `__repr__`, and `__eq__` based on class attributes. We can use **decorators** to:

1. **Track** or **modify** a data class’s behavior (class decorators),
2. **Add** logic to **methods** within a data class (method decorators).


In [29]:
# Add logic to methods within a data class (method decorators).

import functools
from dataclasses import dataclass

def log_method_call(func):
    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
        print(f"[DEBUG] Method {func.__name__} called on {type(self).__name__}")
        return func(self, *args, **kwargs)
    return wrapper

@dataclass
class BankAccount:
    owner: str
    balance: float = 0.0

    @log_method_call
    def deposit(self, amount: float) -> None:
        self.balance += amount

    @log_method_call
    def withdraw(self, amount: float) -> None:
        if amount > self.balance:
            raise ValueError("Insufficient funds.")
        self.balance -= amount


acct = BankAccount("Alice", 100)
acct.deposit(50)
# [DEBUG] Method deposit called on BankAccount
acct.withdraw(20)
# [DEBUG] Method withdraw called on BankAccount

print(acct)

[DEBUG] Method deposit called on BankAccount
[DEBUG] Method withdraw called on BankAccount
BankAccount(owner='Alice', balance=130)


Here:
1. `@log_method_call` logs each method call before running the actual logic.
2. We target **only** specific methods within the data class.

---

# 3. Decorators + Generics

## 3.1 Decorators for Generic Functions

When we say “generics,” we typically reference typed containers or functions that rely on `TypeVar`. A decorator can handle a generic function, but note that many Python type checkers might require you to carefully handle the function signature.

### Example 5: Generic Decorator for Re-Usable Logic


In [30]:
from typing import TypeVar, Generic, Callable
import functools

T = TypeVar("T")
R = TypeVar("R")

def ensure_positive_result(func: Callable[[T], R]) -> Callable[[T], R]:
    """
    A generic decorator that ensures the result of the wrapped function is positive.
    Raises ValueError if not.
    """
    @functools.wraps(func)
    def wrapper(param: T) -> R:
        result = func(param)
        if isinstance(result, (int, float)) and result < 0:
            raise ValueError(f"Result must be positive, got {result}")
        return result
    return wrapper

@ensure_positive_result
def half(x: float) -> float:
    return x / 2.0

print(half(10.0))  # 5.0

5.0


In [32]:
# print(half(-1.0)) # Raises ValueError

**Explanation**:
1. We define `ensure_positive_result(func)` as a function that returns a `Callable[[T], R]`, using generics for the input/return types.  
2. The decorator checks if the returned value is numeric and negative; if so, it raises an error.  
3. This can work for any function that returns a numeric result—**type checkers** will enforce that the function’s signature matches `(T) -> R`.

# 4. Putting It All Together

Decorators can:
1. **Wrap functions** to add logging, caching, validation, or other cross-cutting concerns.
2. **Wrap classes** (including **data classes**) to alter how they’re initialized, track instances, or impose constraints.
3. **Use generics** so they stay flexible for different function signatures or data class type parameters.

**Key Best Practices**:
- **Use `functools.wraps`** when decorating functions. This preserves important metadata.  
- **Document your decorators** carefully, especially if they’re generic or if they modify class behavior in unexpected ways.  
- **Mind the order** of multiple decorators. Python applies them **top-down**. For instance,  
  ```python
  @decoratorA
  @decoratorB
  def my_func(): ...
  ```  
  is equivalent to  
  ```python
  my_func = decoratorA(decoratorB(my_func))
  ```


- **Test thoroughly** if your decorator modifies function or class behavior that can affect state or type expectations.

---

# Final Summary

1. **Function Decorators**: Great for adding common logic (logging, validation, caching) around any callable.  
2. **Class Decorators**: Especially useful for data classes, letting you inject or modify their behavior upon instantiation.  
3. **Generics**: Allow decorators to remain type-safe across a variety of parameter and return types, critical in large, type-hinted Python codebases.  
4. **DataClasses + Decorators**: You can decorate individual methods inside a data class or decorate the class as a whole for advanced initialization/validation.  

This **comprehensive** approach—blending **decorators**, **data classes**, and **generics**—helps you write more **modular**, **scalable**, and **maintainable** Python applications, including advanced use cases in AI, web backends, or any data-driven system. Experiment, refine, and enjoy the powerful metaprogramming capabilities Python offers!